In [1]:
#Librerias a utilizar
import requests
from bs4 import BeautifulSoup
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import pandas as pd
from pandas.io.json import json_normalize

# Web scraping

# API Parsing

In [2]:
#variables para poder conectar a la API de spotifi, son provistos por developer spotify
CLIENT_ID = "2b7f57a2e7d74e43b0a0771a99fbb470"
CLIENT_SECRET = "219cfd4ed31349d5b3175e3526de4921"

#Autentificacion y uso de las variables de arriba
credentials = oauth2.SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)

#Creacion de los token de autenticación de spotify
token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)


<class 'spotipy.client.Spotify'>


In [18]:
#ayuda con el metodo 
help(spotify.user_playlist_tracks)

Help on method user_playlist_tracks in module spotipy.client:

user_playlist_tracks(user, playlist_id=None, fields=None, limit=100, offset=0, market=None) method of spotipy.client.Spotify instance
    Get full details of the tracks of a playlist owned by a user.
    
    Parameters:
        - user - the id of the user
        - playlist_id - the id of the playlist
        - fields - which fields to return
        - limit - the maximum number of tracks to return
        - offset - the index of the first track to return
        - market - an ISO 3166-1 alpha-2 country code.



In [4]:
#usamos el id de usuario y el id de la playlist para obtener las canciones y todos los datos que se manejan en la bd de spotify
canciones = spotify.user_playlist_tracks(12101725182, '3LXuqFWKxpJGn6yoTvDAOG')
dataframe = pd.DataFrame(canciones)

In [26]:
#pruebas para llegar a la url de la cancion, track name y popularity
dataframe2 = json_normalize(canciones['items'])

In [41]:
#mas pruebas
dataframe2[['track.href', 'tr']]

0    https://api.spotify.com/v1/tracks/02woMWxtxB5A...
1    https://api.spotify.com/v1/tracks/2vg16RThWkdr...
2    https://api.spotify.com/v1/tracks/1ZY1PqizIl78...
3    https://api.spotify.com/v1/tracks/1Ax3ybwNz6KB...
4    https://api.spotify.com/v1/tracks/7zKxoK67ddk6...
5    https://api.spotify.com/v1/tracks/4PpuZIMmeng6...
6    https://api.spotify.com/v1/tracks/7EWbEYuEDzr5...
Name: track.href, dtype: object